### Chinook database (SQLite) - Answering Business Questions using SQL

The following code connect our Jupyter Notebook to our database file:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

Getting the tables of the database

In [2]:
%%sql

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Obtaining the information from the customer table

In [3]:
%%sql
PRAGMA table_info(customer);

 * sqlite:///chinook.db
Done.


cid,name,type,notnull,dflt_value,pk
0,customer_id,INTEGER,1,None,1
1,first_name,NVARCHAR(40),1,None,0
2,last_name,NVARCHAR(20),1,None,0
3,company,NVARCHAR(80),0,None,0
4,address,NVARCHAR(70),0,None,0
5,city,NVARCHAR(40),0,None,0
6,state,NVARCHAR(40),0,None,0
7,country,NVARCHAR(40),0,None,0
8,postal_code,NVARCHAR(10),0,None,0
9,phone,NVARCHAR(24),0,None,0


#### Chinook schema 

<img src="chinook-schema.svg">

### 1. Which genres sell the most tracks in the USA?

In [4]:
%%sql

WITH  invoice_USA  AS (SELECT invoice_id
                         FROM invoice
                        WHERE billing_country = 'USA'),
     
invoice_line_genre AS (SELECT i.invoice_id,
                              i.quantity,
                              g.name 
                         FROM invoice_line AS i
                        INNER JOIN track AS t ON t.track_id = i.track_id
                        INNER JOIN genre AS g ON g.genre_id = t.genre_id),
    
units_sold_by_genre AS (SELECT name AS genre,
                               SUM(quantity) AS units
                          FROM invoice_USA  
                         INNER JOIN invoice_line_genre ON invoice_line_genre.invoice_id = invoice_USA.invoice_id    
                         GROUP BY name),
     
        total_units AS (SELECT SUM(units) AS total
                          FROM units_sold_by_genre)
    
SELECT genre,
       units,
       ROUND(CAST(units AS Float) * 100 / (SELECT total FROM total_units), 2) AS percentage   
  FROM units_sold_by_genre
 ORDER BY units DESC    

 * sqlite:///chinook.db
Done.


genre,units,percentage
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


### 2. Sales support agent and sales

In [5]:
%%sql

WITH sales_support_customers AS (SELECT e.first_name || ' ' || e.last_name AS sales_support,
                                        e.hire_date,
                                        e.city,
                                        c.customer_id
                                   FROM employee AS e
                                  INNER JOIN customer AS c ON c.support_rep_id = e.employee_id),

     customer_invoice AS (SELECT i.customer_id,
                                 SUM(i.total) AS total
                            FROM customer AS c
                           INNER JOIN invoice AS i ON i.customer_id = c.customer_id 
                           GROUP BY i.customer_id)
                        
SELECT sales_support, 
       hire_date,
       city,
       SUM(total) AS total_sales
  FROM sales_support_customers AS s
 INNER JOIN customer_invoice AS c ON c.customer_id = s.customer_id
 GROUP BY sales_support
                        


 * sqlite:///chinook.db
Done.


sales_support,hire_date,city,total_sales
Jane Peacock,2017-04-01 00:00:00,Calgary,1731.5099999999998
Margaret Park,2017-05-03 00:00:00,Calgary,1584.0000000000002
Steve Johnson,2017-10-17 00:00:00,Calgary,1393.92


### 3. Analyze the sales data of customers from each different country.

In [6]:
%%sql

SELECT country, COUNT(customer_id) AS Number_customers
  FROM customer
 GROUP BY country 
 ORDER BY Number_customers

 * sqlite:///chinook.db
Done.


country,Number_customers
Argentina,1
Australia,1
Austria,1
Belgium,1
Chile,1
Denmark,1
Finland,1
Hungary,1
Ireland,1
Italy,1


Because there are several countries with only one customer, we should group these customers as "Other" in your analysis. The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.

In [7]:
%%sql

WITH data AS (SELECT country, COUNT(customer_id) AS Number_customers, 
                     CASE
                         WHEN COUNT(customer_id) = 1 THEN 'Other'
                         ELSE country 
                      END AS Aux_country
                FROM customer
               GROUP BY country
               ORDER BY Number_customers)

SELECT Aux_country AS country,
       SUM(Number_customers) AS Number_customers
  FROM data
 GROUP BY Aux_country
 ORDER BY Number_customers DESC

 * sqlite:///chinook.db
Done.


country,Number_customers
Other,15
USA,13
Canada,8
Brazil,5
France,5
Germany,4
United Kingdom,3
Czech Republic,2
India,2
Portugal,2


In [8]:
%%sql

WITH data AS (SELECT CASE
                         WHEN COUNT(DISTINCT(c.customer_id)) = 1 THEN 'Other'
                         ELSE country 
                     END AS country,
                     COUNT(DISTINCT(c.customer_id)) AS number_customers,
                     SUM(total) AS total_sales,
                     COUNT(invoice_id) AS number_orders
                FROM customer AS c
               INNER JOIN invoice AS i ON i.customer_id = c.customer_id
               GROUP BY country),
     
     country_data AS (SELECT country, 
                             SUM(number_customers) AS number_customers,
                             SUM(total_sales) AS total_sales,
                             SUM(number_orders) AS number_orders,
                             CASE
                                 WHEN country = 'Other' THEN 1             
                                 ELSE 0
                              END AS aux_sort
                        FROM data
                       GROUP BY country
                       ORDER BY total_sales DESC)

SELECT country,
       number_customers,
       ROUND(total_sales, 2) AS total_sales,
       ROUND(total_sales / number_customers, 2) AS avg_sales_per_customer,
       ROUND(total_sales / number_orders, 2) AS avg_order_value
  FROM country_data
 ORDER BY aux_sort DESC

 * sqlite:///chinook.db
Done.


country,number_customers,total_sales,avg_sales_per_customer,avg_order_value
Other,15,1094.94,73.0,7.45
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.65,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.57,8.72


### 4.  What sells the most, individual songs from albums or full albums?

Are there tracks duplicated in the invoices?

In [9]:
%%sql

WITH data AS (SELECT invoice_id || '_' || i.track_id || '_' || t.album_id AS code
                FROM invoice_line AS i
               INNER JOIN track AS t ON t.track_id = i.track_id)

SELECT COUNT(*) AS total_number_invoice_id_line,        
       COUNT(DISTINCT(code)) AS number_invoice_id_line_no_duplicated

FROM data

 * sqlite:///chinook.db
Done.


total_number_invoice_id_line,number_invoice_id_line_no_duplicated
4757,4745


In the next query we'll see that in some invoices we have tracks from differents albums.

In [10]:
%%sql

/*In the next query we'll see that in some invoices 
we have tracks from differents albums.*/

SELECT invoice_id, t.album_id, CAST(invoice_id AS Text) || '-' || CAST(t.album_id AS Text) AS  invoice_album
  FROM invoice_line AS i
 INNER JOIN track AS t ON t.track_id = i.track_id  
 GROUP BY invoice_album
 ORDER BY invoice_id
LIMIT 10

 * sqlite:///chinook.db
Done.


invoice_id,album_id,invoice_album
1,91,1-91
2,134,2-134
2,190,2-190
2,20,2-20
2,215,2-215
2,218,2-218
2,322,2-322
2,34,2-34
2,39,2-39
2,66,2-66


In [11]:
%%sql

WITH invoice_album AS (SELECT invoice_id, 
                              t.album_id, 
                              invoice_id || '_' || t.album_id AS invoice_album,
                              COUNT(DISTINCT(invoice_id || '_' || i.track_id || '_' || t.album_id)) AS number_tracks_in_invoice_same_album                              
                         FROM invoice_line AS i
                        INNER JOIN track AS t ON t.track_id = i.track_id  
                        GROUP BY invoice_album
                        ORDER BY invoice_id),

     data AS (SELECT *, 
                     (SELECT COUNT(track_id)
                        FROM track AS t
                       WHERE t.album_id = ia.album_id) AS number_tracks_in_album,
                  
                     CASE 
                         WHEN (SELECT COUNT(track_id)
                         FROM track AS t
                        WHERE t.album_id = ia.album_id) = number_tracks_in_invoice_same_album THEN 'yes'
                     ELSE 'no'
                     END AS full_album  
              FROM invoice_album AS ia)
                             
SELECT full_album, 
       COUNT(full_album) AS number,
       ROUND(CAST(100 * COUNT(full_album) AS Float) / (SELECT COUNT(*) FROM data),2) AS percentage
  FROM data AS d
 GROUP BY full_album

 * sqlite:///chinook.db
Done.


full_album,number,percentage
no,2869,93.24
yes,208,6.76


Another way of obtain the same data

In [12]:
%%sql

WITH invoice_album AS (SELECT invoice_id, 
                              t.album_id, 
                              invoice_id || '_' || t.album_id AS invoice_album                            
                         FROM invoice_line AS i
                        INNER JOIN track AS t ON t.track_id = i.track_id  
                        GROUP BY invoice_album
                        ORDER BY invoice_id),
                     
              data AS (SELECT *,  
                              CASE
                                 WHEN(-- tracks in the album
                                      SELECT t.track_id
                                      FROM track AS t
                                      WHERE t.album_id = i.album_id

                                 EXCEPT 

                                 SELECT il.track_id 
                                   FROM invoice_line AS il 
                                  INNER JOIN track AS t ON il.track_id = t.track_id
                                  WHERE il.invoice_id = i.invoice_id AND t.album_id = i.album_id
                                 ) IS NULL
                                
                                 AND
                               
                                 (SELECT il.track_id 
                                    FROM invoice_line AS il 
                                   INNER JOIN track AS t ON il.track_id = t.track_id
                                   WHERE il.invoice_id = i.invoice_id AND t.album_id = i.album_id

                                 EXCEPT 

                                 SELECT t.track_id 
                                   FROM track AS t
                                  WHERE t.album_id = i.album_id
                                 ) IS NULL
                              THEN "yes"
                              ELSE "no"
                               END AS "full_album"
                         FROM invoice_album AS i)

SELECT full_album, 
       COUNT(full_album) AS number,
       ROUND(CAST(100 * COUNT(full_album) AS Float) / (SELECT COUNT(*) FROM data),2) AS percentage
  FROM data 
 GROUP BY full_album

 * sqlite:///chinook.db
Done.


full_album,number,percentage
no,2869,93.24
yes,208,6.76


### 5. How many tracks have been purchased vs not purchased?

In [13]:
%%sql
                     
WITH purchased AS (-- all the tracks
                   SELECT track_id    
                     FROM track 

                INTERSECT

                   --purchased tracks
                   SELECT DISTINCT(track_id)
                     FROM invoice_line),
     
  no_purchased AS (SELECT track_id
                       FROM track  

                   EXCEPT

                   SELECT DISTINCT(track_id)
                     FROM invoice_line),
    
    union_data AS (SELECT *, 
                          'purchased' AS 'track' 
                     FROM purchased

                    UNION

                   SELECT *,  
                          'no purchased' AS 'track' 
                     FROM no_purchased)
        
SELECT track,
       COUNT(track) AS total,
       ROUND(CAST(100 * COUNT(track) AS Float) / (SELECT COUNT(*) FROM union_data), 2) AS percentage 
  FROM union_data
 GROUP BY track

 * sqlite:///chinook.db
Done.


track,total,percentage
no purchased,1697,48.44
purchased,1806,51.56
